In [2]:
# 读抓包文件得到数据包
# pip install scapy
from scapy.all import *
import socket
from scapy.utils import PcapNgReader 
packets = PcapNgReader('1.pcapng').read_all(count=-1)
data={}
time={}
for i in range(1, len(packets)):
    if 'UDP' in packets[i]:
        # 获取UDP包的内容
        source = (packets[i]['IP'].src,packets[i]['UDP'].sport)
        payload = packets[i]['UDP'].payload.load
        t = packets[i].time 
        
        if source not in time:time[source]=[]
        time[source].append((t,len(data.get(source,b''))))
        if source not in data:data[source]=b''
        data[source]=data[source]+payload
print(f'source: {data.keys()}')

source: dict_keys([('192.168.66.13', 8899), ('192.168.66.11', 8899)])


In [6]:
import numpy as np
from scipy.interpolate import interp1d
import pandas
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

def merge(x:dict,y:dict):
    ret = {}
    for i in x:
        ret[i] = x[i]
    for i in y:
        ret[i] = ret[i] + y[i]
    return ret
def Parse_2121(data,time_list):
    ret = {
        "timestamp":[],
        "rolling":[],
        "TagID":[],
        "AnchorIdx":[],
        "IQ":[],
        "SYNCTime":[],
        "REPLYTime":[]
    }
    for i,t in zip(data,time_list):

        包头,报文长度 = struct.unpack("2H",i[:4])
        报文正文 = i[4:报文长度+2]
        校验=i[报文长度+2:报文长度+4]
        
        Type,Length= struct.unpack("2H",报文正文[:4])
        Value = 报文正文[4:]
        
        Source_ID,流水号,AnchorCount,Tag_ID0=struct.unpack("4H",Value[:8])
        M = AnchorCount
        N = (Length-8-M*8)//6
        if M != N:
            continue # 没收齐的不管
        peakpos,I,Q = np.frombuffer(Value[8:8+6*N],dtype=np.int16).reshape((3,-1),order='F')
        IQ = I+1j*Q
        peakpos=peakpos.astype(np.uint16)
        SYNCTime,REPLYTime = np.frombuffer(Value[8+6*N:8+6*N+8*M],dtype=np.uint32).reshape((2,-1),order='F')
        # preamble,totalLen,type,length = struct.unpack()
        ret = merge(ret,{
            "timestamp":M*[t],
            "rolling":M*[流水号],
            "TagID":M*[Tag_ID0],
            "AnchorIdx":list(range(M)),
            "IQ":IQ.tolist(),
            "SYNCTime":SYNCTime.tolist(),
            "REPLYTime":REPLYTime.tolist()
        })
    return pandas.DataFrame(ret)
tmp = b''
def send(data: bytes):
    start = 0
    cnt = 0
    while True:
        tmp = data[start: start + 1000]
        if not tmp:
            break
        s.sendto(tmp, ("127.0.0.1", 9999))
        start += 1000
        cnt += 1
        print(len(tmp), cnt)
result = []
for device in time:
    send(data[device])
    tmp += data[device]
    # print(len(tmp))
    timeslice = np.array([[float(i)for i in j] for j in time[device] ])
    timeslice[:,0]=timeslice[:,0]-timeslice[0,0]
    dataslice = np.frombuffer(data[device],dtype=np.uint8)
    preambles = np.where((dataslice[:-1] == 0xF5) & (dataslice[1:] == 0x5a ))[0]
    f = interp1d(timeslice[:, 1], timeslice[:, 0], kind='nearest', bounds_error=False, fill_value="extrapolate")
    time_list = f(preambles)
    lengths = dataslice[preambles[:,None]+[[2,3]]]@[1,256]+2
    print(lengths)
    types = dataslice[preambles[:,None]+[[4,5]]]@[1,256]
    typeofInterest = types==0x2121
    time_list=time_list[typeofInterest]
    lengths=lengths[typeofInterest]
    preambles=preambles[typeofInterest]
    frames = [dataslice[preambles[i]:preambles[i]+lengths[i]].tobytes() for i in range(len(preambles))]
    result.append(Parse_2121(frames,time_list))
result = pandas.concat(result).sort_values('rolling')
result.to_csv('test.csv')


1000 1
1000 2
1000 3
1000 4
1000 5
1000 6
1000 7
1000 8
1000 9
1000 10
1000 11
1000 12
1000 13
1000 14
1000 15
1000 16
1000 17
1000 18
1000 19
1000 20
1000 21
1000 22
1000 23
1000 24
1000 25
1000 26
1000 27
1000 28
1000 29
1000 30
1000 31
1000 32
1000 33
1000 34
1000 35
1000 36
1000 37
1000 38
1000 39
1000 40
1000 41
1000 42
1000 43
1000 44
1000 45
1000 46
1000 47
1000 48
1000 49
1000 50
1000 51
1000 52
1000 53
1000 54
1000 55
1000 56
1000 57
1000 58
1000 59
1000 60
1000 61
1000 62
1000 63
1000 64
1000 65
1000 66
1000 67
1000 68
1000 69
1000 70
1000 71
1000 72
1000 73
1000 74
1000 75
1000 76
1000 77
1000 78
1000 79
1000 80
1000 81
1000 82
1000 83
1000 84
1000 85
1000 86
1000 87
1000 88
1000 89
1000 90
1000 91
1000 92
1000 93
1000 94
1000 95
1000 96
1000 97
1000 98
1000 99
1000 100
1000 101
1000 102
1000 103
1000 104
1000 105
1000 106
1000 107
1000 108
1000 109
1000 110
1000 111
1000 112
1000 113
1000 114
1000 115
1000 116
1000 117
1000 118
1000 119
1000 120
1000 121
1000 122
1000 123
1

In [5]:
# 有两个基站和两个标签之间交叉的全部4组数据才是有效数据
# 计算有效数据数量
isvalid = []
for i in np.unique(result['rolling']):
    isvalid.append(len(result[result['rolling'] == i]))
Groundtruth_validRate = (np.array(isvalid)==4).mean()
print(Groundtruth_validRate)
from collections import Counter

rolling1 = set({k: v for k, v in Counter(result['rolling']).items() if v == 4}.keys())


0.9007675438596491


In [73]:
# 计算上位机保存的数据的有效数据数量
result2=pandas.read_csv('CIR相位集中上传.csv')
isvalid2 = []
for i in np.unique(result2['rolling']):
    isvalid2.append(len(result2[result2['rolling'] == i]))
validRate = (np.array(isvalid2)==4).mean()
print(validRate)
rolling2 = set({k: v for k, v in Counter(result2['rolling']).items() if v == 4}.keys())
print(rolling1 - rolling2)


0.9007675438596491
set()


In [ ]:
# 确认我数据不存在两来源的数据时间相差过久 导致汇聚时被滤掉的情况
Groundtruth_dt=[]
for i in np.unique(result['rolling']):
    temp=np.array(result[result['rolling'] == i]['timestamp'])
    Groundtruth_dt.append(temp.max()-temp.min())
print(np.percentile(Groundtruth_dt,50),np.percentile(Groundtruth_dt,95),np.percentile(Groundtruth_dt,99))
# 50%数据在0.45秒内到齐
# 95%数据在0.95秒内到齐
# 99%数据在1.04秒内到齐
# 之前设计需求里写的"如果缓存中有某个滚码,且这个滚码最后一次填入数据到现在超过0.2秒,则这个滚码的数据已经收完了,可以取出继续下一步"
# 应该改成1.5秒汇聚间隔 最好改成可配置的
# 此外,如果出现同一个滚码的数据间隔较长时间到来两波,预期行为是数据被记录到文件两次,而不是丢弃一部分数据:
#  0时刻 滚码1 标签1
#  0时刻 滚码1 标签2
#  1时刻 滚码2 标签1
#  1时刻 滚码2 标签2
#  1时刻 滚码2 标签3
#  1时刻 滚码2 标签4
#  2时刻 滚码1 标签3
#  2时刻 滚码1 标签4
# 预期的记录:
#  滚码1 标签1
#  滚码1 标签2
#  滚码2 标签1
#  滚码2 标签2
#  滚码2 标签3
#  滚码2 标签4
#  滚码1 标签3  # 这两条不能丢弃 可以乱序
#  滚码1 标签4
#  滚码1 标签1  

In [30]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

data = open('20240428_145916.bin', 'rb').read()
def send(data: bytes):
    start = 0
    cnt = 0
    while True:
        tmp = data[start: start + 1000]
        if not tmp:
            break
        s.sendto(tmp, ("127.0.0.1", 9999))
        start += 1000
        cnt += 1
        print(len(tmp), cnt)
send(data)

1000 1
1000 2
1000 3
1000 4
1000 5
1000 6
1000 7
1000 8
1000 9
1000 10
1000 11
1000 12
1000 13
1000 14
1000 15
1000 16
1000 17
1000 18
1000 19
1000 20
1000 21
1000 22
1000 23
1000 24
1000 25
1000 26
1000 27
1000 28
1000 29
1000 30
1000 31
1000 32
1000 33
1000 34
1000 35
1000 36
1000 37
1000 38
1000 39
1000 40
1000 41
1000 42
1000 43
1000 44
1000 45
1000 46
1000 47
1000 48
1000 49
1000 50
1000 51
1000 52
1000 53
1000 54
1000 55
1000 56
1000 57
1000 58
1000 59
1000 60
1000 61
1000 62
1000 63
1000 64
1000 65
1000 66
1000 67
1000 68
1000 69
1000 70
1000 71
1000 72
1000 73
1000 74
1000 75
1000 76
1000 77
1000 78
1000 79
1000 80
1000 81
1000 82
1000 83
1000 84
1000 85
1000 86
1000 87
1000 88
1000 89
1000 90
1000 91
1000 92
1000 93
1000 94
1000 95
1000 96
1000 97
1000 98
1000 99
1000 100
1000 101
1000 102
1000 103
1000 104
1000 105
1000 106
1000 107
1000 108
1000 109
1000 110
1000 111
1000 112
1000 113
1000 114
1000 115
1000 116
1000 117
1000 118
1000 119
1000 120
1000 121
1000 122
1000 123
1